# Marketing Campaign Analysis

## Background

### About the Business

Company A's clients have substantial debt and because of unexpected hardship are no longer able to make their minimum monthly on their debts. clients enroll with Company A and rather than making payments to creditors, they make affordable deposits into a dedicated account with Company A, who uses these funds to negotiate with the creditors to settle the client's outstanding debt. Company A then collects fees from the client for the that was settled. 

Company A earns fees for each account it successfully negotiates a settlement agreement; the number of settlement agreements Company A can negotiate is proportional to the monthly deposited amount. 

### Project Details

Company A ran a recent marketing campaign to promote the value proposition of how the debt relief program helps people achieve financial freedom; the cost of this campaign was $5 million. The goal of this analysis is to **show marketing, sales and operations the success of the campaign**. Specifically, the analysis includes: 

- A quantitative assessment of whether the marketing campaign was successful.
- Recommended adjustments to the campaign strategy to improve performance.

### Defining Success
Given our available data, these are the possible success factors that will be considered:

- Decrease in incomplete scheduled deposits
    - How to measure: Time-based comparison of incomeplete vs. complete scheduled deposits for clients who had accounts prior to the campaign.  
- Increase in deposit frequency
    - How to measure: Time-based comparison of the number of different deposit frequencies for clients who had accounts prior to the campaign.  
- Increase in average amount deposited each month per client
    - How to measure: Time-based comparison of average summed mothly deposits for clients who had accounts prior to the campaign. 
- Increase in client base: 
    - How to measure: Time-based comparison of total client count during the 5 months of data. 

Note: We will not consider an increase in extra/non-recurrent deposits as a success because Company A can only negotiate based on the monthly deposited amount (recurring deposits). As such, while extra deposits can make up for a shortfall, they do not positively impact Company A's bottom line.  

### Data Overview

There are three datasets provided for the analysis; each is already cleaned and prepared for analysis.  

**client_data.csv: Fictional clients**

| Column Name | Description |
|---|---|
client_id|Unique client ID|
client_geographical_region|Client geo-location (U.S. Census definitions)|
client_residence_status|Client residence status|
client_age|Client age|


**deposit_data.csv: Client deposit behavior**

| Column Name | Description |
|---|---|
client_id|Unique client ID|
deposit_type|Scheduled vs. Actual deposit|
deposit_amount|Deposit amount|
deposit_cadence|Frequency of deposit (scheduled)|
deposit_date|Deposit date|


**calendar_data.csv: Calendar reference table**

| Column Name | Description |
|---|---|
|gregorian_date|Gregorian calendar date|
|month_name|Month relative to data set|

Notes: 

- Month 1 and 2 are pre-campaign
- Month 3 is the campaign
- Month 4 and 5 are post-campaign

Assumptions: 

- There is no seasonality in the results
- The campaign spend was distributed evenly across Month 3 (i.e., spend on the first day is the same as spend on the last day)

## Analysis

In [251]:
# import packages
import numpy as np
import pandas as pd
import datetime
import plotly.express as px

### Data Wrangling

For this first section, I'm taking a quick look at the provided data. Since the data is already cleaned, we're assuming that there are not duplicate client accounts and all deposits are authentic representations rather than data entry errors.  

In the initial review, it's clear that there are *generally* two representations for deposits: a scheduled deposit then the actual deposit. As such, we're going to create dummy variables for the two deposit types then group the deposits together so there is only one sample per deposit. This will create the following options for each deposit:

|Actual|Scheduled|Description|  
|---|---|---|
|True|True|A scheduled deposit is completed|  
|True|False|A scheduled deposit is cancelled and not completed|  
|False|True|An unscheduled deposit is made|

Note: It is possible for someone to make multiple deposits of the same amount on the same day. As such, we are not dropping deposits that may appear to be duplicated. 

At the end of this section, we will have a sample representing an individual deposit. The columns are as follows:

|Column Name|Description|Value Type/Values|
|---|---|---|
|client_id| Unique client ID|Integer|
|deposit_amount| Amount of Deposit|Decimal |
|deposit_cadence| Frequency of Deposit|Categorical: Bi-Weekly, Extra, Monthly |
|deposit_date| Gregorian date of deposit| datetime|
|actual_deposit| If the deposit was completed | Boolean: True/False|
|scheduled_deposit| If the deposit was scheduled | Boolean: True/False|
|month_name| Month relative to data set|Categorical: 1, 2, 3, 4, 5 |
|client_geographical_region|Client geo-location (U.S. Census definitions)|Categorical: Midwest, West, South, Northeast |
|client_residence_status|Client Residence Status |Categorical: Rent, Own|
|client_age| Client's Age|Integer |


In [252]:
# read in data
clients = pd.read_csv('data/client_data.csv')
deposits = pd.read_csv('data/deposit_data.csv')
calendar = pd.read_csv('data/calendar_data.csv')

# return basic details on dataframes
df_group = [clients, deposits,calendar]
for i in df_group:
    print(i.head())
    print('Total rows: ',i.shape[0])
    print('')

         client_id  ... client_age
0  538839486596724  ...         91
1  321708286091707  ...         83
2  848531901757235  ...         84
3  854405182328779  ...         83
4  769102176031316  ...         85

[5 rows x 4 columns]
Total rows:  46347

         client_id       deposit_type  ...  deposit_cadence deposit_date
0  446495122764671     Actual Deposit  ...          Monthly   2019-10-23
1  446495122764671     Actual Deposit  ...          Monthly   2019-09-23
2  446495122764671  Scheduled Deposit  ...          Monthly   2019-09-23
3  446495122764671  Scheduled Deposit  ...          Monthly   2019-10-23
4  446495122764671  Scheduled Deposit  ...          Monthly   2019-06-23

[5 rows x 5 columns]
Total rows:  480394

  gregorian_date month_name
0     2019-06-01    Month 1
1     2019-06-02    Month 1
2     2019-06-03    Month 1
3     2019-06-04    Month 1
4     2019-06-05    Month 1
Total rows:  153

         client_id  ... client_age
0  538839486596724  ...         91
1  32170828

In [253]:
# verify types of deposits
deposits.deposit_type.unique()

array(['Actual Deposit', 'Scheduled Deposit'], dtype=object)

In [254]:
# get dummies for deposit types
df = pd.get_dummies(data=deposits, prefix='', prefix_sep='', columns=['deposit_type'])
print(df.head())
print('Total rows: ',df.shape[0])

         client_id  deposit_amount  ... Actual Deposit Scheduled Deposit
0  446495122764671           303.0  ...           True             False
1  446495122764671           303.0  ...           True             False
2  446495122764671           303.0  ...          False              True
3  446495122764671           303.0  ...          False              True
4  446495122764671           303.0  ...          False              True

[5 rows x 6 columns]
Total rows:  480394
         client_id  deposit_amount  ... Actual Deposit Scheduled Deposit
0  446495122764671           303.0  ...           True             False
1  446495122764671           303.0  ...           True             False
2  446495122764671           303.0  ...          False              True
3  446495122764671           303.0  ...          False              True
4  446495122764671           303.0  ...          False              True

[5 rows x 6 columns]
Total rows:  480394


In [255]:
# group scheduled/actual deposits for matching
df = df.groupby(['client_id', 'deposit_amount', 'deposit_cadence', 'deposit_date'], 
    as_index=False).agg({'Actual Deposit': 'max', 'Scheduled Deposit': 'max'})
print(df.head())
print('Total Rows: ',df.shape[0])

      client_id  deposit_amount  ... Actual Deposit Scheduled Deposit
0  146046305811           247.0  ...          False              True
1  146046305811           247.0  ...           True              True
2  146046305811           247.0  ...           True              True
3  146046305811           247.0  ...           True              True
4  146046305811           247.0  ...           True              True

[5 rows x 6 columns]
Total Rows:  256143
      client_id  deposit_amount  ... Actual Deposit Scheduled Deposit
0  146046305811           247.0  ...          False              True
1  146046305811           247.0  ...           True              True
2  146046305811           247.0  ...           True              True
3  146046305811           247.0  ...           True              True
4  146046305811           247.0  ...           True              True

[5 rows x 6 columns]
Total Rows:  256143


In [256]:
# merge dataframes
df = pd.merge(df, calendar, left_on='deposit_date', right_on='gregorian_date', 
    how='inner', suffixes=('',''))
df = pd.merge(df, clients, on='client_id', how='inner', suffixes=('',''))

# clean columns
df['month_name'] = df['month_name'].str.extract(r'(\d+)').astype(int)
df = df.rename(columns={'month_name': 'month', 'Actual Deposit': 'actual_deposit', 
    'Scheduled Deposit': 'scheduled_deposit'})
df = df.drop(['gregorian_date'], axis=1)

df.head()

,client_id,deposit_amount,deposit_cadence,deposit_date,actual_deposit,scheduled_deposit,month,client_geographical_region,client_residence_status,client_age
0,146046305811,247.0,Biweekly,2019-06-05,False,True,1,Midwest,Rent,42
1,146046305811,247.0,Biweekly,2019-06-19,True,True,1,Midwest,Rent,42
2,146046305811,247.0,Biweekly,2019-07-03,True,True,2,Midwest,Rent,42
3,146046305811,247.0,Biweekly,2019-07-17,True,True,2,Midwest,Rent,42
4,146046305811,247.0,Biweekly,2019-07-31,True,True,2,Midwest,Rent,42


In [257]:
# update column types
df = df.astype({'deposit_cadence':'category', 'month':'category', 
    'client_geographical_region':'category', 'client_residence_status':'category'})
df['deposit_date'] = pd.to_datetime(df['deposit_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256143 entries, 0 to 256142
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   client_id                   256143 non-null  int64         
 1   deposit_amount              256143 non-null  float64       
 2   deposit_cadence             256143 non-null  category      
 3   deposit_date                256143 non-null  datetime64[ns]
 4   actual_deposit              256143 non-null  bool          
 5   scheduled_deposit           256143 non-null  bool          
 6   month                       256143 non-null  category      
 7   client_geographical_region  256143 non-null  category      
 8   client_residence_status     256143 non-null  category      
 9   client_age                  256143 non-null  int64         
dtypes: bool(2), category(4), datetime64[ns](1), float64(1), int64(2)
memory usage: 9.3 MB
<class 'pandas.cor

### Initial Results 

Following that, we'll create a new df with clients who already had accounts in month 1 of the dataset since the first three success methods are dependent on the existing client base. We'll also filter out "Extra" deposits because company profits are tied to recurring deposits. 

In [258]:
# get a list of clients who had accounts in month 1
month_1_clients = df[df['month'] == 1]
month_1_clients = month_1_clients.client_id.unique()

# df only containing clients from month 1
df2 = df[df['client_id'].isin(month_1_clients)]

# focus only on biweekly and monthly deposits
df2 = df2[df2['deposit_cadence'].isin(['Biweekly', 'Monthly'])]

# setup dataframe specific to completed deposits
df3 = df2[df2['actual_deposit'] == True]

# return results
print('Existing Customers')
print(df2.head())
print('Total rows: ',df2.shape[0])
print('')
print('Completed Deposits by Existing Customers')
print(df3.head())
print('Total rows: ',df3.shape[0])


Existing Customers
      client_id  deposit_amount  ... client_residence_status client_age
0  146046305811           247.0  ...                    Rent         42
1  146046305811           247.0  ...                    Rent         42
2  146046305811           247.0  ...                    Rent         42
3  146046305811           247.0  ...                    Rent         42
4  146046305811           247.0  ...                    Rent         42

[5 rows x 10 columns]
Total rows:  195343

Completed Deposits by Existing Customers
      client_id  deposit_amount  ... client_residence_status client_age
1  146046305811           247.0  ...                    Rent         42
2  146046305811           247.0  ...                    Rent         42
3  146046305811           247.0  ...                    Rent         42
4  146046305811           247.0  ...                    Rent         42
5  146046305811           247.0  ...                    Rent         42

[5 rows x 10 columns]
Total row

#### Decrease in incomplete scheduled deposits

    - How to measure: Time-based comparison of incomeplete vs. complete scheduled deposits for clients who had accounts prior to the campaign.  

In [259]:
# establish df for incomplete and complete transactions
status = df2[['deposit_date', 'month', 'actual_deposit']]
status = status.groupby(['deposit_date', 'actual_deposit'], as_index=False, observed=True).size()
status

,deposit_date,actual_deposit,size
0,2019-06-01,False,256
1,2019-06-01,True,2034
2,2019-06-02,False,87
3,2019-06-02,True,673
4,2019-06-03,False,137
...,...,...,...
301,2019-10-29,True,318
302,2019-10-30,False,274
303,2019-10-30,True,1123
304,2019-10-31,False,162


In [260]:
fig1 = px.line(status, x="deposit_date", y="size", color='actual_deposit')
fig1.show()

#### Increase in deposit frequency

    - How to measure: Time-based comparison of the number of different deposit frequencies for clients who had accounts prior to the campaign.  

In [261]:
# establish df for different cadences
cadence = df3[['deposit_date', 'month', 'deposit_cadence']]
cadence = cadence.groupby(['deposit_date', 'month', 'deposit_cadence'], as_index=False, observed=True).size()
cadence

,deposit_date,month,deposit_cadence,size
0,2019-06-01,1,Biweekly,286
1,2019-06-01,1,Monthly,1748
2,2019-06-02,1,Biweekly,47
3,2019-06-02,1,Monthly,626
4,2019-06-03,1,Biweekly,275
...,...,...,...,...
301,2019-10-29,5,Monthly,278
302,2019-10-30,5,Biweekly,233
303,2019-10-30,5,Monthly,890
304,2019-10-31,5,Biweekly,291


In [262]:
fig2 = px.line(cadence, x="deposit_date", y="size", color='deposit_cadence')
fig2.show()

#### Increase in average amount deposited each month per client

    - How to measure: Time-based comparison of average summed mothly deposits for clients who had accounts prior to the campaign. 

In [263]:
# establish df for summed monthly deposits
amount = df3[['month', 'client_id', 'deposit_amount']]
amount = amount.groupby(['client_id', 'month'], as_index=False, observed=True).agg({'deposit_amount': 'sum'})
amount = amount.groupby(['month'], as_index=False, observed=True).agg({'deposit_amount': 'mean'})
amount

,month,deposit_amount
0,1,470.162524
1,2,472.484384
2,3,465.540002
3,4,468.167816
4,5,470.272147


#### Increase in client base

- How to measure: Time-based comparison of total client count during the 5 months of data. 

In [266]:
client_count = df2[['client_id', 'month']]
client_count = client_count.drop_duplicates()
client_count = df2[['month']]
client_count = client_count.groupby(['month'], as_index=False, observed=True).size()
client_count.head()


,month,size
0,1,41167
1,2,40200
2,3,39770
3,4,36419
4,5,37787


In [ ]:
fig4 = px.line(df2, x="deposit_date", y="size", color='deposit_cadence')
fig4.show()